In [10]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
import time

In [21]:
os.chdir('C:\\Users\\shic\\Desktop\\Project\\venture')
pd.set_option('display.max_seq_items', None)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [22]:
ven = pd.read_csv('ven_com_2020.csv',encoding='cp949')
ven.shape

(2500, 357)

In [23]:
ven.columns

Index(['아이디', '구분_법인/개인', '가중치', '[쿼터] 지역', '[쿼터] 업종', '[쿼터] 규모',
       'A1-4. 벤처확인 유형', 'A1-5. 회사주소_시(도)', 'A1-5. 회사주소_구(군)',
       'A1-6. KSIC코드_10차', 'A1-6-1. 1단계 업종분류_대분류', 'A1-6-2. 2단계 업종분류_중분류',
       'A1-6-3. 3단계 업종분류_소분류', 'A2-1. 창업년도', 'A2-2. 기업성장단계', 'A2-3. 제조 및 비제조',
       'A2-3. 일부 외주인 경우, 아웃소싱 비중(%)', 'A2-4. 스마트공장 도입의향', 'A2-5. 상장여부 및 향후계획',
       'A2-5. 현재 상장기업_상장년도', 'A2-5. 현재 상장기업_상장시장', 'B1-1. 현 대표이사와 창업자 동일여부',
       'B1-2. 대표이사 성별', 'B1-3. 대표이사 연령(현재)', 'B1-4. 대표이사 최종학력(현재)',
       'B1-4-1. 대표이사 전공', 'B1-5. 대표이사 직전 근무지', 'B1-6. 대표이사 이전 업무경력분야(1)',
       'B1-6. 대표이사 이전 업무경력분야(2)', 'B1-6. 대표이사 이전 업무경력분야(3)',
       'B1-6. 대표이사 이전 업무경력분야(4)', 'B1-6. 대표이사 이전 업무경력분야(5)',
       'B1-6. 대표이사 이전 업무경력분야(6)', 'B1-6. 대표이사 이전 업무경력분야(7)',
       'B1-6. 대표이사 이전 업무경력분야(8)', 'B2-1. 창업자 성별', 'B2-2. 창업 당시 창업자 연령',
       'B2-3. 창업 당시 창업자 최종학력', 'B2-3-1. 창업 당시 창업자 전공', 'B2-4. 창업 직전 근무지',
       'B2-5. 창업 이전 업무경력분야(1)', 'B2-5. 창업 이전 업무경력분야(2)',
       'B2-5. 창업 이전 업무경력분야(3)', 'B

In [27]:
ven.head()

,아이디,구분_법인/개인,가중치,[쿼터] 지역,[쿼터] 업종,[쿼터] 규모,A1-4. 벤처확인 유형,A1-5. 회사주소_시(도),A1-5. 회사주소_구(군),A1-6. KSIC코드_10차,A1-6-1. 1단계 업종분류_대분류,A1-6-2. 2단계 업종분류_중분류,A1-6-3. 3단계 업종분류_소분류,A2-1. 창업년도,A2-2. 기업성장단계,A2-3. 제조 및 비제조,"A2-3. 일부 외주인 경우, 아웃소싱 비중(%)",A2-4. 스마트공장 도입의향,A2-5. 상장여부 및 향후계획,A2-5. 현재 상장기업_상장년도,A2-5. 현재 상장기업_상장시장,B1-1. 현 대표이사와 창업자 동일여부,B1-2. 대표이사 성별,B1-3. 대표이사 연령(현재),B1-4. 대표이사 최종학력(현재),B1-4-1. 대표이사 전공,B1-5. 대표이사 직전 근무지,B1-6. 대표이사 이전 업무경력분야(1),B1-6. 대표이사 이전 업무경력분야(2),B1-6. 대표이사 이전 업무경력분야(3),B1-6. 대표이사 이전 업무경력분야(4),B1-6. 대표이사 이전 업무경력분야(5),B1-6. 대표이사 이전 업무경력분야(6),B1-6. 대표이사 이전 업무경력분야(7),B1-6. 대표이사 이전 업무경력분야(8),B2-1. 창업자 성별,B2-2. 창업 당시 창업자 연령,B2-3. 창업 당시 창업자 최종학력,B2-3-1. 창업 당시 창업자 전공,B2-4. 창업 직전 근무지,B2-5. 창업 이전 업무경력분야(1),B2-5. 창업 이전 업무경력분야(2),B2-5. 창업 이전 업무경력분야(3),B2-5. 창업 이전 업무경력분야(4),B2-5. 창업 이전 업무경력분야(5),B2-5. 창업 이전 업무경력분야(6),B2-5. 창업 이전 업무경력분야(7),B2-5. 창업 이전 업무경력분야(8),B2-6-1. 창업방식,B2-6-2. 단독창업 여부,"B2-6-2. 공동/팀 창업시, 창업자 포함 멤버 수(명)",B2-6-3. 창업자 역할,B2-6-4. 공동창업자 역할,B2-7. 창업 당시 목표시장,B2-8. 창업 당시 창업자 실무경험 년수(년),B2-9. 창업자 과거창업 경험 여부,B2-9-1. 과거창업 결과(횟수)_성공,B2-9-1. 과거창업 결과(횟수)_실패,B2-9-1. 과거창업 결과(횟수)_합계,C1-1. 2019년 정규직_남성,C1-1. 2019년 정규직_여성,C1-1. 2019년 정규직_합계,C1-2. 2019년 비정규직_남성,C1-2. 2019년 비정규직_여성,C1-2. 2019년 비정규직_합계,C1-3. 2019년 총 종사자_남성,C1-3. 2019년 총 종사자_여성,C1-3. 2019년 총 종사자_합계,C2-1. 직종별 인력구성_관리,C2-1. 직종별 인력구성_생산,C2-1. 직종별 인력구성_영업,C2-1. 직종별 인력구성_R&D,C2-1. 직종별 인력구성_기타,C2-1. 직종별 인력구성_합계,C2-1. 직종별 부족인원_관리,C2-1. 직종별 부족인원_생산,C2-1. 직종별 부족인원_영업,C2-1. 직종별 부족인원_R&D,C2-1. 직종별 부족인원_기타,C2-1. 직종별 부족인원_합계,C3. 학력별 인력구성_박사,C3. 학력별 인력구성_석사,C3. 학력별 인력구성_대졸,C3. 학력별 인력구성_전문대졸,C3. 학력별 인력구성_고졸이하,C3. 학력별 인력구성_합계,C4. 신규인력 채용의사,C4. 신규채용 예상인력_2020년 하반기,C4. 신규채용 예상인력_2021년,C4. 신규채용 예상인력_합계,C4-1. 신규채용 희망학력_박사,C4-1. 신규채용 희망학력_석사,C4-1. 신규채용 희망학력_대졸,C4-1. 신규채용 희망학력_전문대졸,C4-1. 신규채용 희망학력_고졸이하,C4-1. 신규채용 희망학력_합계,C4-2. 신규인력 채용계획 없는 이유,C5-1. 현재 구인하기 어려운 직무(1순위),C5-1. 현재 구인하기 어려운 직무(2순위),C5-2. 현재 구인하기 어려운 직급(1순위),C5-2. 현재 구인하기 어려운 직급(2순위),D1. 지분구조(%)_창업자,D1. 지분구조(%)_임직원,D1. 지분구조(%)_가족,D1. 지분구조(%)_벤처캐피털 및 기관투자자,D1. 지분구조(%)_엔젤 및 액셀러레이터,D1. 지분구조(%)_개인투자자,D2-1. 2019년 자산(백만원),D2-2. 2019년 부채(백만원),D2-3. 2019년 자본(백만원),D3-1. 2019년 매출액(백만원),D3-2. 2019년 영업이익(백만원),D3-3. 2019년 금융비용(백만원),D3-4. 2019년 당기순이익(백만원),D4. 매출구조(%)_B2B(대기업 또는 대기업그룹사 소속사),D4. 매출구조(%)_B2B(대기업에 납품하는 1·2차 벤더),D4. 매출구조(%)_B2B(중소기업 및 벤처기업),D4. 매출구조(%)_B2B(중견기업),D4. 매출구조(%)_B2C(소비자 매출),D4. 매출구조(%)_B2G(정부 포함 공공부문 매출),D4. 매출구조(%)_해외,D5-1. R&D자금(출연금)_경험여부,D5-1. R&D자금(출연금)_금액(백만원),D5-2. 융자(정책자금)_경험여부,D5-2. 융자(정책자금)_금액(백만원),D5-3. 보증서지원_경험여부,D5-3. 보증서지원_금액(백만원),D5-4. 기타지원_경험여부,D5-4. 기타지원_금액(백만원),D5-5. 합계_경험여부,D5-5. 합계_금액(백만원),D6. 신규자금 조달규모(백만원),D6. 신규자금 조달방법(%)_IPO,D6. 신규자금 조달방법(%)_벤처캐피털·엔젤투자,D6. 신규자금 조달방법(%)_회사채 발행,D6. 신규자금 조달방법(%)_정부정책지원금,D6. 신규자금 조달방법(%)_은행 등 일반금융,D6. 신규자금 조달방법(%)_기타,D7. 투자처별 투자액(백만원)_국내설비,D7. 투자처별 투자액(백만원)_국내R&D,D7. 투자처별 투자액(백만원)_해외,D7. 투자처별 투자액(백만원)_기타,D7. 투자처별 투자액(백만원)_합계,D8. 스톡옵션 활용 상황,D8-1. 스톡옵션 실시 목적,D8-2. 스톡옵션 미활용 이유,D9. 1~2년내 법인전환 의사,E1. 주력기술과 4차 산업혁명 관련 여부,E1-1. 주력제품과 관련성이 큰 기술(1),E1-1. 주력제품과 관련성이 큰 기술(2),E1-1. 주력제품과 관련성이 큰 기술(3),E1-1. 주력제품과 관련성이 큰 기술(4),E1-1. 주력제품과 관련성이 큰 기술(5),E1-1. 주력제품과 관련성이 큰 기술(6),E1-1. 주력제품과 관련성이 큰 기술(7),E1-1. 주력제품과 관련성이 큰 기술(8),E1-1. 주력제품과 관련성이 큰 기술(9),E1-1. 주력제품과 관련성이 큰 기술(10),E1-1. 주력제품과 관련성이 큰 기술(11),E1-1. 주력제품과 관련성이 큰 기술(12),E1-1. 주력제품과 관련성이 큰 기술(13),E1-1. 주력제품과 관련성이 큰 기술(14),E1-1. 주력제품과 관련성이 큰 기술(15),E1-1. 주력제품과 관련성이 큰 기술(16),E1-1. 주력제품과 관련성이 큰 기술(17),E1-1. 주력제품과 관련성이 큰 기술(18),E1-1. 주력제품과 관련성이 큰 기술(19),E1-1. 주력제품과 관련성이 큰 기술(20),E2-1. 현재 등록보유 산업재산권(건)_특허권,E2-1. 현재 등록보유 산업재산권(건)_실용신안권,E2-1. 현재 등록보유 산업재산권(건)_디자인권,E2-1. 현재 등록보유 산업재산권(건)_상표권,E2-1. 현재 등록보유 산업재산권(건)_해외,E2-1. 현재 등록보유 산업재산권(건)_합계,E2-2. 현재 출원중 산업재산권(건)_특허권,E2-2. 현재 출원중 산업재산권(건)_실용신안권,E2-2. 현재 출원중 산업재산권(건)_디자인권,E2-2. 현재 출원중 산업재산권(건)_상표권,E2-2. 현재 출원중 산업재산권(건)_해외,E2-2. 현재 출원중 산업재산권(건)_합계,E3. 획득하고 있는 해외특허나 국제규격,E4. 제품 및 서비스 구조,E5. 세계 유일기술 보유여부,E6-1. 세계 최고수준 기업과 비교시 기술력 수준,E6-2. 국내 최고수준 기업과 비교시 기술력 수준,E7. 기술연구소나 연구개발 전담부서 보유여부,F1-1. 경쟁사 대비 현황_개발역량,F1-2. 경쟁사 대비 현황_제조역량,F1-3. 경쟁사 대비 현황_마케팅역량,F2-1. 경쟁전략초점_원가우위 수준,F2-2. 경

In [24]:
venc = pd.read_csv('ven_com_2020_code.csv',encoding='cp949')
venc.shape

(454, 3)

In [25]:
venc.columns

Index(['구분', '코드값', '코드설명'], dtype='object')

In [35]:
pd.Series(venc.구분.value_counts().index.sort_values())

0                            A1-4. 벤처확인유형
1                            A2-2. 기업성장단계
2                          A2-3. 제조 및 비제조
3                        A2-4. 스마트공장 도입의향
4                      A2-5. 상장여부 및 향후 계획
5                      A2-5. 현재 상장기업_상장시장
6                B1-1. 현재 대표이사와 창업자 동일 여부
7                           B1-2. 대표이사 성별
8                       B1-3. 대표이사 연령(현재)
9                         B1-4-1. 대표이사 전공
10                    B1-4. 대표이사 최종학력(현재)
11                      B1-5. 대표이사 직전 근무지
12                   B1-6. 대표이사 이전 업무경력분야
13                           B2-1. 창업자 성별
14                     B2-2. 창업 당시 창업자 연령
15                   B2-3-1. 창업 당시 창업자 전공
16                   B2-3. 창업 당시 창업자 최종학력
17                        B2-4. 창업 직전 근무지
18                     B2-5. 창업 이전 업무경력분야
19                           B2-6-1. 창업방식
20                        B2-6-2. 단독창업 여부
21                         B2-6-3. 창업자 역할
22                       B2-6-4. 공동창업자 역할
23                       B2-7. 창업 